In [1]:
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F

from leafy.loader import ImageLoader
from leafy.trainloader import TrainLoader

data_folder = Path("./images")
loader = ImageLoader(data_folder=data_folder)
trainloader = TrainLoader(loader)

# class_distribution = loader.get_better_class_distribution()

Initiated loader on folder /home/joep/Code/Leafliction/images. Found 7233 images.


In [2]:
trainloader.gt


tensor([[0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        ...,
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]])

torch.float32

In [3]:
from tqdm.notebook import tqdm
classes = set()

for im, c in tqdm(loader):
    classes.add(c)

print(im.shape)
print(classes)

  0%|          | 0/7233 [00:00<?, ?it/s]

torch.Size([3, 256, 256])
{'Grape_Esca', 'Grape_healthy', 'Apple_scab', 'Apple_rust', 'Grape_spot', 'Grape_Black_rot', 'Apple_healthy', 'Apple_Black_rot'}


In [3]:
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import Module

class BasicClassifier(Module):
    def __init__(self, input_shape, classes):
        super(BasicClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.conv4 = nn.Conv2d(32, 16, 5)
        self.conv5 = nn.Conv2d(16, 8, 5)
        self.conv6 = nn.Conv2d(8, 4, 5)
        # self.convs = [self.conv1 ,self.conv2 ,self.conv3 ,self.conv4 ,self.conv5 ,self.conv6]
        self.convs = [self.conv1, self.conv2, self.conv5, self.conv6]
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(4 * 12 * 12, len(classes))
        
    def forward(self, x):
        for conv in self.convs:
            x = self.pool(F.relu(conv(x)))
        
        x = self.fc(x.view(-1))
        return F.softmax(x)


net = BasicClassifier([3, 256, 256], classes)

y = net(im)
print(y)


NameError: name 'classes' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHWQ
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class UNet(nn.Module):
    def __init__(self, n_channels = 1, n_classes = 1, width = 64, depth = 4, bilinear=False):
        super(UNet, self).__init__()
        self.down  = nn.ModuleList()
        self.up    = nn.ModuleList()

        for i in range(depth):
            self.down.append(Down(width * 2**i    , width * 2**(i+1)))
            self.up.insert(0,  Up(width * 2**(i+1), width * 2**i    , bilinear))

        self.down.insert(0, DoubleConv(n_channels, width))
        self.outc = OutConv(width, n_classes)


    def forward(self, x):
        values = [x]

        for layer in self.down:
            values.append(layer(values[-1]))

        for layer in self.up:
            first  = values.pop(-1)
            second = values.pop(-1)
            values.append(layer(first, second))

        return self.outc(values[-1])


unet_layer_classes = {
    'DoubleConv' : DoubleConv,
    'Down'       : Down,
    'Up'         : Up,
    'OutConv'    : OutConv,
    'UNet'       : UNet,
}